In [1]:
import time
import sklearn
import numpy as np
import pandas as pd

import sys
sys.path.append("..")
from baggingPU import BaggingClassifierPU
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
def print_cm(cm, labels, hide_zeroes=False, hide_diagonal=False, hide_threshold=None):
    columnwidth = max([len(x) for x in labels]) + 4
    empty_cell = " " * columnwidth
    print("    " + empty_cell, end=' ')
    for label in labels:
        print("%{0}s".format(columnwidth) % 'pred_' + label, end=" ")
    print()

    for i, label1 in enumerate(labels):
        print("    %{0}s".format(columnwidth) % 'true_' + label1, end=" ")
        for j in range(len(labels)):
            cell = "%{0}.1f".format(columnwidth) % cm[i, j]
            if hide_zeroes:
                cell = cell if float(cm[i, j]) != 0 else empty_cell
            if hide_diagonal:
                cell = cell if i != j else empty_cell
            if hide_threshold:
                cell = cell if cm[i, j] > hide_threshold else empty_cell
            if cell:
                print(cell, end=" ")
        print()

# import data

In [3]:
#df_raw = pd.read_csv('../data/w-dependence.csv')

#df_raw = pd.read_csv('../data/1place-dependence.csv')
df_raw = pd.read_csv('../data/1-place2.csv')
# df_raw = pd.read_csv('../data/w-related.csv')

df_raw['label'] = df_raw['label'].astype("int")
print(df_raw.label.value_counts())
print('Has null values', df_raw.isnull().values.any())

1    10000
0    10000
Name: label, dtype: int64
Has null values False


In [5]:
df_raw.head(10)

,p1,p2,label
0,1,0,1
1,1,2,1
2,1,4,1
3,1,6,1
4,1,8,1
5,1,10,1
6,1,12,1
7,1,14,1
8,1,16,1
9,1,18,1


In [6]:
def random_undersampling(tmp_df, TARGET_LABEL):
    df_majority = tmp_df[tmp_df[TARGET_LABEL] == 0]
    df_minority = tmp_df[tmp_df[TARGET_LABEL] == 1]

    df_majority_downsampled = resample(df_majority, 
                                       replace=False,              
                                       n_samples=len(df_minority), 
                                       random_state=None)        

    df_downsampled = pd.concat([df_majority_downsampled, df_minority])

    print("Undersampling complete!")
    print(df_downsampled[TARGET_LABEL].value_counts())
    return df_downsampled

In [7]:
df_downsampled = random_undersampling(df_raw, 'label')
df_downsampled = df_downsampled.sample(frac=1) 
df_downsampled = df_downsampled.reset_index() 
df_downsampled = df_downsampled.drop(columns=['index']) 

Undersampling complete!
1    10000
0    10000
Name: label, dtype: int64


In [8]:
df_downsampled.head(10)

,p1,p2,label
0,1,7084,1
1,1,13700,1
2,1,16926,1
3,1,3331,0
4,1,17023,0
5,1,9803,0
6,1,1926,1
7,1,19994,1
8,1,17975,0
9,1,10328,1


In [9]:
x_data = df_raw.iloc[:,:-1]
y_data = df_raw.iloc[:,-1]

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=7)

x_train_input = pd.concat([x_train,y_train],axis=1)
x_test_input = pd.concat([x_test,y_test],axis=1)

print(len(x_train_input))
print(len(x_test_input))

print(x_train_input.label.value_counts())
print('Has null values', x_train_input.isnull().values.any())

print(x_test_input.label.value_counts())
print('Has null values', x_test_input.isnull().values.any())

16000
4000
1    8077
0    7923
Name: label, dtype: int64
Has null values False
0    2077
1    1923
Name: label, dtype: int64
Has null values False


In [10]:
x_train_input.head(10)

,p1,p2,label
8118,1,16236,1
10171,1,343,0
9399,1,18798,1
12669,1,5339,0
18809,1,17619,0
13869,1,7739,0
3320,1,6640,1
14689,1,9379,0
13087,1,6175,0
15992,1,11985,0


In [11]:
df = x_train_input.copy()

NON_LBL = [c for c in df.columns if c != 'label']
X = df[NON_LBL]
y = df['label']

y_orig = y.copy()

hidden_size = 5000
y.loc[
    np.random.choice(
        y[y == 1].index, 
        replace = False, 
        size = hidden_size
    )
] = 0

In [12]:
pd.Series(y).value_counts()

0    12923
1     3077
Name: label, dtype: int64

In [13]:
print('- %d samples and %d features' % (X.shape))
print('- %d positive out of %d total before hiding labels' % (sum(df_downsampled.label), len(df_downsampled.label)))
print('- %d positive out of %d total after hiding labels' % (sum(y), len(y)))

- 16000 samples and 2 features
- 10000 positive out of 20000 total before hiding labels
- 3077 positive out of 16000 total after hiding labels


# Trainning directly

In [16]:
print('Training SVM model ...')

from sklearn import svm

model = svm.SVC(kernel='rbf')

model.fit(X, y)

print('Done')

Training SVM model ...
Done


In [17]:
print('---- {} ----'.format('SVM'))
print(print_cm(sklearn.metrics.confusion_matrix(y_orig, model.predict(X)), labels=['negative', 'positive']))
print('')
print('Precision: ', precision_score(y_orig, model.predict(X)))
print('Recall: ', recall_score(y_orig, model.predict(X)))
print('Accuracy: ', accuracy_score(y_orig, model.predict(X)))
print('f1_score: ', f1_score(y_orig, model.predict(X)))

---- SVM ----
                        pred_negative        pred_positive 
           true_negative       7923.0          0.0 
           true_positive       8077.0          0.0 
None

Precision:  0.0
Recall:  0.0
Accuracy:  0.4951875
f1_score:  0.0


# Training by bagging

In [18]:
print('Training bagging classifier...')

pu_start = time.perf_counter()
model = BaggingClassifierPU(svm.SVC(kernel='rbf'),
                         n_estimators = 50, 
                         n_jobs = -1, 
                         max_samples = sum(y)
                        )
model.fit(X, y)
pu_end = time.perf_counter()
print('Done!')
print('Time:', pu_end - pu_start)

Training bagging classifier...
Done!
Time: 136.36012683099398


In [19]:
#train data
print('---- {} ----'.format('PU Bagging'))
print(print_cm(sklearn.metrics.confusion_matrix(y_orig, model.predict(X)), labels=['negative', 'positive']))
print('')
print('Precision: ', precision_score(y_orig, model.predict(X)))
print('Recall: ', recall_score(y_orig, model.predict(X)))
print('Accuracy: ', accuracy_score(y_orig, model.predict(X)))
print('f1_score: ', f1_score(y_orig, model.predict(X)))

---- PU Bagging ----
                        pred_negative        pred_positive 
           true_negative       3518.0       4405.0 
           true_positive       3504.0       4573.0 
None

Precision:  0.5093562040543551
Recall:  0.5661755602327597
Accuracy:  0.5056875
f1_score:  0.5362650249193786


In [62]:
#print wrong predictions
y_pre = model.predict(X)
y_orig_index = y_orig.index.tolist()

FN_index = []
FT_index = []

for i in range(len(y_orig)):
    if y_orig.iloc[i] == 1 and y_pre[i] == 0 :
        FN_index.append(y_orig_index[i])
    if y_orig.iloc[i] == 0 and y_pre[i] == 1 :
        FT_index.append(y_orig_index[i])
        
print("False Negtive:")
print(X.loc[FN_index])
print("False Positive:")
print(X.loc[FT_index])

False Negtive:
Empty DataFrame
Columns: [p1, p2, p3, p4, p5, p6, p7, p8]
Index: []
False Positive:
Empty DataFrame
Columns: [p1, p2, p3, p4, p5, p6, p7, p8]
Index: []


In [63]:
#test data
print('---- {} ----'.format('PU Bagging'))
print(print_cm(sklearn.metrics.confusion_matrix(y_test, model.predict(x_test)), labels=['negative', 'positive']))
print('')
print('Precision: ', precision_score(y_test, model.predict(x_test)))
print('Recall: ', recall_score(y_test, model.predict(x_test)))
print('Accuracy: ', accuracy_score(y_test, model.predict(x_test)))
print('f1_score: ', f1_score(y_test, model.predict(x_test)))

---- PU Bagging ----
                        pred_negative        pred_positive 
           true_negative       2077.0          0.0 
           true_positive          0.0       1923.0 
None

Precision:  1.0
Recall:  1.0
Accuracy:  1.0
f1_score:  1.0


In [64]:
#print wrong predictions
y_test_pre = model.predict(x_test)
y_test_index = y_test.index.tolist()

FN_test_index = []
FT_test_index = []

for i in range(len(y_test)):
    if y_test.iloc[i] == 1 and y_test_pre[i] == 0 :
        FN_test_index.append(y_test_index[i])
    if y_test.iloc[i] == 0 and y_test_pre[i] == 1 :
        FT_test_index.append(y_test_index[i])

print("False Negtive:")
print(x_test.loc[FN_test_index])
print("False Positive:")
print(x_test.loc[FT_test_index])

False Negtive:
Empty DataFrame
Columns: [p1, p2, p3, p4, p5, p6, p7, p8]
Index: []
False Positive:
Empty DataFrame
Columns: [p1, p2, p3, p4, p5, p6, p7, p8]
Index: []


In [65]:
a = pd.read_csv('../data/wdepenpos5000.csv')

a['label'] = a['label'].astype("int")
print(a.label.value_counts())
print('Has null values', a.isnull().values.any())

1    5000
Name: label, dtype: int64
Has null values False


In [66]:
x_data = a.iloc[:,:-1]

x_data['p3'] += 10000
x_data['p1'] += 1

y_data = a.iloc[:,-1]
y_data[:] = 0

print(x_data)
print(y_data)

      p1  p2     p3  p4  p5    p6  p7  p8
0      1   0  18277   0   0  9751   1   0
1      1   0  18946   0   0  7585   1   0
2      1   1  19359   0   0  3291   0   0
3      1   0  14894   0   1  3736   0   0
4      1   0  16136   1   0  4114   0   0
...   ..  ..    ...  ..  ..   ...  ..  ..
4995   2   0  17813   0   0  5727   0   0
4996   1   0  16953   1   0  9351   0   0
4997   2   0  15887   0   0  2023   0   0
4998   1   1  13187   0   0  7671   0   0
4999   1   0  18488   0   0  2968   1   0

[5000 rows x 8 columns]
0       0
1       0
2       0
3       0
4       0
       ..
4995    0
4996    0
4997    0
4998    0
4999    0
Name: label, Length: 5000, dtype: int64


In [67]:
print('---- {} ----'.format('PU Bagging'))
#print(print_cm(sklearn.metrics.confusion_matrix(y_data, model.predict(x_data)), labels=['negative', 'positive']))
print('')
print('Precision: ', precision_score(y_data, model.predict(x_data)))
print('Recall: ', recall_score(y_data, model.predict(x_data)))
print('Accuracy: ', accuracy_score(y_data, model.predict(x_data)))
print('f1_score: ', f1_score(y_data, model.predict(x_data)))

---- PU Bagging ----

Precision:  0.0
Recall:  0.0
Accuracy:  0.0
f1_score:  0.0


In [68]:
#print wrong predictions
y_data_pre = model.predict(x_data)
y_data_index = y_data.index.tolist()

FN_data_index = []
FT_data_index = []

for i in range(len(y_data)):
    if y_data.iloc[i] == 1 and y_data_pre[i] == 0 :
        FN_data_index.append(y_data_index[i])
    if y_data.iloc[i] == 0 and y_data_pre[i] == 1 :
        FT_data_index.append(y_data_index[i])

print("False Negtive:")
print(x_data.loc[FN_data_index])
print("False Positive:")
print(x_data.loc[FT_data_index])

False Negtive:
Empty DataFrame
Columns: [p1, p2, p3, p4, p5, p6, p7, p8]
Index: []
False Positive:
      p1  p2     p3  p4  p5    p6  p7  p8
0      1   0  18277   0   0  9751   1   0
1      1   0  18946   0   0  7585   1   0
2      1   1  19359   0   0  3291   0   0
3      1   0  14894   0   1  3736   0   0
4      1   0  16136   1   0  4114   0   0
...   ..  ..    ...  ..  ..   ...  ..  ..
4995   2   0  17813   0   0  5727   0   0
4996   1   0  16953   1   0  9351   0   0
4997   2   0  15887   0   0  2023   0   0
4998   1   1  13187   0   0  7671   0   0
4999   1   0  18488   0   0  2968   1   0

[5000 rows x 8 columns]
